In [1]:
from selenium import webdriver
from parsel import Selector

In [2]:
driver = webdriver.Chrome(executable_path='C:/Users/dogan/Documents/becode.org/BXL-Bouman-2.22/Content/00.projects/01.scraping/chromedriver.exe')

driver.get('https://www.immoweb.be/en')

driver.find_element_by_xpath('//*[@id="searchBoxSubmitButton"]').click()

In [4]:
sel = Selector(text=driver.page_source) 

xpath_houeses = '//*[@id="main-content"]/li//h2//a/@href'
houses_url = sel.xpath(xpath_houeses).extract()
print(len(houses_url))

30
